# Classification Neural Network

So far, we have primarily worked with regression neural networks. This is only one of the possibilities. Often, we don't want to predict an actual value, instead predicting *what* a grid, set of values, or object is showing. 

In this notebook, we will work to create an incredibly simple neural network classifier: has it rained in the last hour?

In [ ]:
%matplotlib inline

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import copy

Load in our ASOS Data

In [ ]:
input_data = pd.read_csv("./al_asos_jul_23_v2.csv")

In [ ]:
input_data

In [ ]:
np.sum(input_data['p01i'].isna())

In [ ]:
# NaNs are bad!
input_data = input_data.dropna(subset='p01i')
np.sum(input_data['p01i'].isna())

We need to create our categories. Here, we will divide into three categories: No rain, Trace rainfall (0.0001 inches), and raining (>0.0001 inches). While text categories are most useful to us humans, the ANN needs enumerated categories (i.e., integers representing string category names)

In [ ]:
input_data['raining_status'] = ''
input_data['raining_status_int'] = np.empty((len(input_data)), dtype=np.int8)

In [ ]:
input_data.loc[ input_data['p01i'] <= 0.0001, 'raining_status'] = 'TRACE'
input_data.loc[ input_data['p01i'] <= 0.0001, 'raining_status_int'] = 1

input_data.loc[ input_data['p01i'] == 0.0, 'raining_status'] = 'DRY'
input_data.loc[ input_data['p01i'] == 0.0, 'raining_status_int'] = 0

input_data.loc[ input_data['p01i'] > 0.0001, 'raining_status'] = 'RAIN'
input_data.loc[ input_data['p01i'] > 0.0001, 'raining_status_int'] = 2


In [ ]:
input_data['raining_status_int'].unique()

We're going to do something that we should have been doing all along, dividing our input data into train, validate, and test. Scikit-learn has a good function for this. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
training_data, test_validation = train_test_split(input_data, test_size=0.25)
testing_data, validation_data = train_test_split(test_validation, test_size=0.5)

While our input layer is the same as it was before (with a shape of 1), our output layer has changed. Instead, we have three outputs. Each of the three outputs corresponds to one of our output categories (Trace, dry, or rain). Inside the output neurons, the value will be a "score" representing the likelihood of each value. These are called *logits*, the math of which we won't cover here. 

In [ ]:
# same input layer as before
input1 = keras.layers.Input(shape=(1, ))
input_norm = keras.layers.Normalization()(input1)
dense1 = keras.layers.Dense(128, input_dim=1, activation=keras.activations.sigmoid)(input_norm)

# 3 outputs, as we will predict the likelyhood of each classification
output = keras.layers.Dense(3, )(dense1)
model10 = keras.models.Model(inputs=[input1], outputs=[output])
# need to use a different loss function - MSE doesn't make sense anymore
# we also want tensorflow to output accuracy as a metric - how accurate are our predictions?
model10.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001, clipnorm=0.0001), 
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
model10.summary()

In [ ]:
model10.fit(training_data['p01i'].values, training_data['raining_status_int'].values, epochs=4, 
            validation_data=(validation_data['p01i'].values, validation_data['raining_status_int'].values),
            batch_size=64)

Okay, we have fit our classification model to data. Great. Now, how do we evaluate?

In [ ]:
# one way: 
model10.evaluate(testing_data['p01i'].values, testing_data['raining_status_int'].values)

Okay, let's make predictions!

In [ ]:
predictions = model10.predict(testing_data['p01i'].values)
predictions

Fascinating. What *are* those?? They are called *logits*, and their math is outside the scope of this class. The bottom line is that we need to convert them to useful probabilities.

In [ ]:
# add a layer onto an existing model
probability_model10 = tf.keras.Sequential([model10, tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model10.predict(testing_data['p01i'].values)
predictions

Much more understandable output!

In [ ]:
# we can get the actual model prediction (absolute value) by:

predictions_categorical = np.argmax(predictions, axis=1)